In [43]:
import pandas as pd
import numpy as np
import re

In [44]:
df = pd.read_csv("data/train.csv")

In [45]:
df.head(10)

,id,timestamp,retweet_count,user_verified,user_statuses_count,user_followers_count,user_friends_count,user_mentions,urls,hashtags,text
0,0,1588696955143,0,False,68460,1101,1226,NaN,NaN,NaN,Smh I give up
1,1,1588464948124,0,False,309,51,202,NaN,NaN,NaN,"Most of us are Human Beings, but I think you m..."
2,2,1588634673360,0,False,3241,1675,2325,NaN,NaN,NaN,"Old dirty tricks Trump, at it again...like we ..."
3,3,1588433158672,0,False,32327,667,304,NaN,NaN,NaN,Seriously..... I worked 86 hours my last check...
4,4,1588582751599,0,False,581,42,127,NaN,NaN,NaN,May ALMIGHTY ALLAH have mercy on us all. Only ...
5,5,1588434563287,0,False,7214,503,1126,NaN,NaN,NaN,They couldn’t care less.
6,6,1588692966869,2,False,372,738,472,NaN,twitter.com/i/web/status/1…,Ethiopia,Extremely valid points being made here 👇🏾 #Eth...
7,7,1588316892450,1,False,2085,3808,153,NaN,twitter.com/i/web/status/1…,NaN,COVID-19 dominated the discussion Tuesday at a...
8,8,1588625905286,0,False,17765,11666,40,NaN,NaN,NaN,BC now has 112 patients on ventilators. 17 of...
9,9,1588604315931,0,False,3086,66,241,NaN,NaN,NaN,a COVID-19 vaccine would be pretty lit ngl


In [21]:
df["month"].isna().sum()

0

In [46]:
# Just convert False to 0 and True to 1.

df["user_verified"] = df["user_verified"].astype(int)

In [47]:
# I don't know why I am supposed to take the modulo, but it works...
# If I don't do that, I get a wrong date in 1970

df['date']  =(df['timestamp']).astype(np.int64) // 10**3

In [48]:
# unit = 's' puts the date in unix format. Necessary to get the good format.

df['date']= pd.to_datetime(df['date'], unit='s')

In [49]:
df["hour"] = df["date"].dt.hour
df["day"] = df["date"].dt.day
df["month"] = df["date"].dt.month

# 0 for Monday
df["weekday"] = df["date"].dt.weekday

# If the day is a weekend or not
df["weekend"] = np.where(np.logical_or(df["weekday"] == 5, df["weekday"] == 6), 1, 0)

In [50]:
df["friends_followers_ratio"] = df["user_friends_count"]/df["user_followers_count"]

In [51]:
df["has_hashtags"] = np.where(pd.notnull(df["hashtags"]), 1, 0)
df["has_mentions"] = np.where(pd.notnull(df["user_mentions"]), 1, 0)
df["has_urls"] = np.where(pd.notnull(df["urls"]), 1, 0)

In [52]:
# Stupid to use this fonction

def counter(word):
    if pd.isna(word):
        return 0
    number = 1
    for s in word:
        if s == ',':
            number += 1
    return number

In [53]:
df["number_of_urls"] = df["urls"].apply(counter)
df["number_of_mentions"] = df["user_mentions"].apply(counter)
df["number_of_hashtags"] = df["hashtags"].apply(counter)

In [35]:
# An auxiliary function that, given a text, separates it
# with commas (ie useful to get a list of hashtags, urls, mentions)

def word_cut(word):
    if pd.isna(word):
        return []
    word_array = np.array([])
    l = 0
    for i,s in enumerate(word):
        if s == ',':
            word_array = np.append(word_array, word[l:i])
            l = i+1
    word_array = np.append(word_array, word[l::])
    return word_array
    

In [36]:
df["urls_list"] = df["urls"].apply(word_cut)
df["hashtags_list"] = df["hashtags"].apply(word_cut)
df["mentions_list"] = df["user_mentions"].apply(word_cut)

In [272]:
# A dataframe with only non_empty Hashtags

#df_bis = df[df['hashtags_list'].map(lambda d: len(d)) > 0]

In [274]:
# Seems simple but takes too long time... I don't know if there is an optimal solution.

# I added .head(100) to see if it works for just a few tweets.
#df["hashtag_popularity"] = df["hashtags_list"].head(100).apply(lambda x : 0 if x == [] else [df_bis["hashtags_list"].apply(lambda x : 1 if word in x else 0).sum() for word in x])

<ipython-input-274-9c7f81c7d868>:4: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  df["hashtag_popularity"] = df["hashtags_list"].head(100).apply(lambda x : 0 if x == [] else [df_bis["hashtags_list"].apply(lambda x : 1 if word in x else 0).sum() for word in x])


In [59]:
df.to_csv('data/partially_treated_train_data.csv', index = False)